In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [66]:
cd '/content/drive/MyDrive/Test_one/mnist_cnn_practice'

/content/drive/MyDrive/Test_one/mnist_cnn_practice


In [12]:
pip install tensorflow_addons

     |████████████████████████████████| 1.1 MB 5.1 MB/s 


In [6]:
import os
import numpy  as np
import pandas as pd

import matplotlib.pyplot as plt
import tensorflow               as tf
# import tensorflow_addons as tfa
import tqdm
import tensorflow.keras.backend as K
from tensorflow                 import keras
from sklearn.model_selection    import train_test_split
import time
tf.random.set_seed(777)


# 학습/검증 데이터 (Raw data) 준비

In [7]:
# Download the mnist dataset using keras
data_train, data_test = tf.keras.datasets.mnist.load_data()

# Parse images and labels
(images_train, labels_train) = data_train
(images_test, labels_test) = data_test

11501568/11490434 [==============================] - 0s 0us/step


In [20]:
images_train[0]

array([[[  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0]],

       [[  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0]],

       [[  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
      

In [8]:
images_train.shape

(60000, 28, 28)

In [ ]:
labels_train[:10]

array([5, 0, 4, 1, 9, 2, 1, 3, 1, 4], dtype=uint8)

In [ ]:
images_train.shape, labels_train.shape

((60000, 28, 28), (60000,))

## 2D reshaping

In [14]:
images_train = images_train.reshape(images_train.shape[0],images_train.shape[1],images_train.shape[2],1)
images_test = images_test.reshape(images_test.shape[0],images_test.shape[1],images_test.shape[2],1)

In [15]:
labels_train_onehot = np.zeros((labels_train.shape[0],len(np.unique(labels_train))))

for i in range (labels_train.shape[0]):
    labels_train_onehot[i,labels_train[i]] = 1

In [16]:
labels_test_onehot  = np.zeros((labels_test.shape[0],len(np.unique(labels_test))))

for i in range (labels_test.shape[0]):
    labels_test_onehot[i,labels_test[i]] = 1

In [17]:
images_train.shape, labels_train_onehot.shape, images_test.shape, labels_test_onehot.shape

((60000, 28, 28, 1), (60000, 10), (10000, 28, 28, 1), (10000, 10))

## 2D-CNN(Convolution Neural Network)

#### CNN 학습 중 일정 epoch 마다 정확도(Accuracy) 표시 함수 생성

In [10]:
earlystopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', min_delta=0.0001, patience=5, verbose=0,
    mode='auto', baseline=None, restore_best_weights=False
)

In [11]:
PrintAccPerEpochs = 1

class AccuracyPerEpoch(keras.callbacks.Callback):
    def on_epoch_end(self, epoch,logs=None):
        keras.callbacks.Callback()
        if epoch%PrintAccPerEpochs == 0:
            print("[{} Epochs] Accuracy : {:.2f}% ".format(epoch, logs["accuracy"]*100))

#### hyperparameter 설정

In [12]:
Epoch = 200

Kenel_S = [(3,3),(4,4),(5,5)]
Dense_N = [12, 18, 21 , 24]
Learn_R = [0.001]

print('Number of case : %d'%(len(Kenel_S)*len(Dense_N)*len(Learn_R)))

Number of case : 12


In [18]:
TrainData = images_train
TrainLabel = labels_train_onehot

TestData = images_test
TestLabel = labels_test_onehot


[CNN_models] 폴더에 accuracy 0.9이상 모델 자동 저장

In [86]:
cnt = 0

Tr_result_temp = np.zeros((len(Kenel_S)*len(Dense_N)*len(Learn_R) , 5))
for i in range(len(Kenel_S)):
    kernelSize = Kenel_S[i]

    for j in range(len(Dense_N)):
        DenseNeuron = Dense_N[j]

        for k in range(len(Learn_R)):
            start_time = time.time()
            learningRate = Learn_R[k]
            epoch_cnt = 0

            ################ CNN 구조 재설계 ################

            def CNN_model(input_data):
                model = keras.Sequential()
                
                model.add(keras.layers.Conv2D(4, kernel_size=kernelSize, strides=1, padding='same', activation='relu',
                                              input_shape=(input_data.shape[1],input_data.shape[2],input_data.shape[3])))           # Convolution layer 1
                
                model.add(keras.layers.MaxPooling2D(pool_size=(2,2), strides=2, padding='same'))                    # Pooling layer 1
                
                model.add(keras.layers.Conv2D(8,  kernel_size=kernelSize, strides=1, padding='same', activation='relu')) # Convolution layer 2
                
                model.add(keras.layers.MaxPooling2D(pool_size=(2,2), strides=2, padding='same'))                    # Pooling layer 2
                
                model.add(keras.layers.Conv2D(16, kernel_size=kernelSize, strides=1, padding='same', activation='relu')) # Convolution layer 3
                
                model.add(keras.layers.MaxPooling2D(pool_size=(2,2), strides=2, padding='same')) 
                
                model.add(keras.layers.Flatten())
                model.add(keras.layers.Dense(units = DenseNeuron, activation='relu'))
                model.add(keras.layers.Dense(units = 10 , activation='softmax'))                                 # Output Layer

                model.compile(optimizer= keras.optimizers.Adam(learning_rate = learningRate),
                              loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])
                return model

            model = CNN_model(TrainData)
            model.summary()

            print('\n\n[Grid search] case No.%d'%(cnt+1))
            print('Kernel size of each conv. layer : ', kernelSize)
            print('Num of neurons in Dense_H layer : %d'%(DenseNeuron))
            print('Learning rate : %.4f'%(learningRate))

            ################ CNN 학습 ################

            print("\n↓↓↓↓↓ Start CNN training ↓↓↓↓↓\n")

            tf.random.set_seed(777)
            History = model.fit(TrainData, TrainLabel, epochs=Epoch, verbose=0,
                                validation_data=(TestData, TestLabel), callbacks=[AccuracyPerEpoch(),earlystopping])

            # 학습완료된 모델 기반 TestData 진단 정확도
            Loss, Accuracy = model.evaluate(TestData,  TestLabel, verbose=0)
            print("\n[Final Epochs] Accuracy : %.2f"%( Accuracy*100))      
            end_time = time.time()          
            print('time per epoch : %.2f'%((end_time-start_time)/Epoch),'sec')
       

            print('\n\n#####################################################\n\n')

            ################ 학습결과 저장 ################

            # Hyperparameter 케이스별 모델성능 저장
            Tr_result_temp[cnt,0] = cnt
            Tr_result_temp[cnt,1] = (kernelSize)[0]
            Tr_result_temp[cnt,2] = DenseNeuron
            Tr_result_temp[cnt,3] = learningRate
            Tr_result_temp[cnt,4] = Accuracy

            cnt+=1

            # 학습완료된 CNN 모델 저장
            model.save("[CNN_models]/CNN" +"_case%d.h5"%(cnt))

            # CNN 학습과정 저장
            # Hist = pd.DataFrame(np.zeros((Epoch,4)))
            # Hist.iloc[:,0] = np.array(History.history['loss']) # Early stopping 넣으니깐 학습과정 loss에서 길이가 안맞아서 일단 학습과정 저장은 전부 빼버렸습니다
            # Hist.iloc[:,1] = np.array(History.history['val_loss'])
            # Hist.iloc[:,2] = np.array(History.history['accuracy'])
            # Hist.iloc[:,3] = np.array(History.history['val_accuracy'])
#             Hist.to_csv("[CNN_history]/CNN_"+ SG_D +"_case%d_history.csv"%(cnt),header=None,index=None)

Tr_result_temp_pd = pd.DataFrame(Tr_result_temp, columns=['Case', 'Kernel_S', 
                                                          'Dense_N', 'Learn_R', 'Accuracy'])
Tr_result_temp_pd.to_csv("[CNN_models]/Training_result.csv", index=None)                


Model: "sequential_69"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_207 (Conv2D)         (None, 28, 28, 4)         40        
                                                                 
 max_pooling2d_207 (MaxPooli  (None, 14, 14, 4)        0         
 ng2D)                                                           
                                                                 
 conv2d_208 (Conv2D)         (None, 14, 14, 8)         296       
                                                                 
 max_pooling2d_208 (MaxPooli  (None, 7, 7, 8)          0         
 ng2D)                                                           
                                                                 
 conv2d_209 (Conv2D)         (None, 7, 7, 16)          1168      
                                                                 
 max_pooling2d_209 (MaxPooli  (None, 4, 4, 16)       

In [19]:
cnt = 0

Tr_result_temp = np.zeros((len(Kenel_S)*len(Dense_N)*len(Learn_R) , 5))
for i in range(len(Kenel_S)):
    kernelSize = Kenel_S[i]

    for j in range(len(Dense_N)):
        DenseNeuron = Dense_N[j]

        for k in range(len(Learn_R)):
            start_time = time.time()
            learningRate = Learn_R[k]
            epoch_cnt = 0

            ################ CNN 구조 재설계 ################

            def CNN_model(input_data):
                model = keras.Sequential()
                
                model.add(keras.layers.Conv2D(4, kernel_size=kernelSize, strides=1, padding='same', activation='relu',
                                              input_shape=(input_data.shape[1],input_data.shape[2],input_data.shape[3])))           # Convolution layer 1
                
                model.add(keras.layers.MaxPooling2D(pool_size=(2,2), strides=2, padding='same'))                    # Pooling layer 1
                
                model.add(keras.layers.Conv2D(8,  kernel_size=kernelSize, strides=1, padding='same', activation='relu')) # Convolution layer 2
                
                model.add(keras.layers.MaxPooling2D(pool_size=(2,2), strides=2, padding='same'))                    # Pooling layer 2
                
                model.add(keras.layers.Conv2D(16, kernel_size=kernelSize, strides=1, padding='same', activation='relu')) # Convolution layer 3
                
                model.add(keras.layers.MaxPooling2D(pool_size=(2,2), strides=2, padding='same')) 
                
                model.add(keras.layers.Flatten())
                model.add(keras.layers.Dense(units = DenseNeuron, activation='relu'))
                model.add(keras.layers.Dense(units = 10 , activation='softmax'))                                 # Output Layer

                model.compile(optimizer= keras.optimizers.Adam(learning_rate = learningRate),
                              loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])
                return model

            model = CNN_model(TrainData)
            model.summary()

            print('\n\n[Grid search] case No.%d'%(cnt+1))
            print('Kernel size of each conv. layer : ', kernelSize)
            print('Num of neurons in Dense_H layer : %d'%(DenseNeuron))
            print('Learning rate : %.4f'%(learningRate))

            ################ CNN 학습 ################

            print("\n↓↓↓↓↓ Start CNN training ↓↓↓↓↓\n")

            tf.random.set_seed(777)
            History = model.fit(TrainData, TrainLabel, epochs=Epoch, verbose=0,
                                validation_data=(TestData, TestLabel), callbacks=[AccuracyPerEpoch(),earlystopping])

            # 학습완료된 모델 기반 TestData 진단 정확도
            Loss, Accuracy = model.evaluate(TestData,  TestLabel, verbose=0)
            print("\n[Final Epochs] Accuracy : %.2f"%( Accuracy*100))      
            end_time = time.time()          
            print("Final epoch:", epoch_cnt)
            print('time per epoch : %.2f'%((end_time-start_time)/Epoch),'sec')
       

            print('\n\n#####################################################\n\n')

            ################ 학습결과 저장 ################

            # Hyperparameter 케이스별 모델성능 저장
            Tr_result_temp[cnt,0] = cnt
            Tr_result_temp[cnt,1] = (kernelSize)[0]
            Tr_result_temp[cnt,2] = DenseNeuron
            Tr_result_temp[cnt,3] = learningRate
            Tr_result_temp[cnt,4] = Accuracy

            cnt+=1

            # 학습완료된 CNN 모델 저장
            model.save("[CNN_models]/CNN" +"_case%d.h5"%(cnt))

            # CNN 학습과정 저장
            # Hist = pd.DataFrame(np.zeros((Epoch,4)))
            # Hist.iloc[:,0] = np.array(History.history['loss']) # Early stopping 넣으니깐 학습과정 loss에서 길이가 안맞아서 일단 학습과정 저장은 전부 빼버렸습니다
            # Hist.iloc[:,1] = np.array(History.history['val_loss'])
            # Hist.iloc[:,2] = np.array(History.history['accuracy'])
            # Hist.iloc[:,3] = np.array(History.history['val_accuracy'])
#             Hist.to_csv("[CNN_history]/CNN_"+ SG_D +"_case%d_history.csv"%(cnt),header=None,index=None)

Tr_result_temp_pd = pd.DataFrame(Tr_result_temp, columns=['Case', 'Kernel_S', 
                                                          'Dense_N', 'Learn_R', 'Accuracy'])
Tr_result_temp_pd.to_csv("[CNN_models]/Training_result.csv", index=None)                


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 4)         40        
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 4)        0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 14, 8)         296       
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 7, 7, 8)          0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 7, 7, 16)          1168      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 4, 4, 16)         0

In [23]:
Tr_result_temp_pd.sort_values('Accuracy',ascending= False)

,Case,Kernel_S,Dense_N,Learn_R,Accuracy
6,6.0,4.0,21.0,0.001,0.9849
8,8.0,5.0,12.0,0.001,0.9843
10,10.0,5.0,21.0,0.001,0.9828
1,1.0,3.0,18.0,0.001,0.9822
2,2.0,3.0,21.0,0.001,0.9814
5,5.0,4.0,18.0,0.001,0.9813
11,11.0,5.0,24.0,0.001,0.9813
3,3.0,3.0,24.0,0.001,0.9811
4,4.0,4.0,12.0,0.001,0.9801
0,0.0,3.0,12.0,0.001,0.9799


In [3]:
Epoch

NameError: ignored

In [1]:
Tr_result_temp_pd.to_csv("[CNN_models]/Training_result.csv", index=None)   # 위에 다 돌아가면 이거 추가로 반드시 돌리기!!!

NameError: ignored

In [80]:
Tr_result_temp_pd

,Case,Kernel_S,Dense_N,Learn_R,Accuracy
0,0.0,3.0,12.0,0.001,0.9462
1,1.0,3.0,18.0,0.001,0.9581
2,2.0,3.0,21.0,0.001,0.9697
3,3.0,3.0,24.0,0.001,0.9729
4,4.0,4.0,12.0,0.001,0.9611
5,5.0,4.0,18.0,0.001,0.9762
6,6.0,4.0,21.0,0.001,0.9763
7,7.0,4.0,24.0,0.001,0.9746
8,8.0,5.0,12.0,0.001,0.9661
9,9.0,5.0,18.0,0.001,0.9767


In [81]:
Tr_result_temp_pd.sort_values('Accuracy',ascending = False)

,Case,Kernel_S,Dense_N,Learn_R,Accuracy
11,11.0,5.0,24.0,0.001,0.9795
9,9.0,5.0,18.0,0.001,0.9767
6,6.0,4.0,21.0,0.001,0.9763
5,5.0,4.0,18.0,0.001,0.9762
7,7.0,4.0,24.0,0.001,0.9746
3,3.0,3.0,24.0,0.001,0.9729
10,10.0,5.0,21.0,0.001,0.9698
2,2.0,3.0,21.0,0.001,0.9697
8,8.0,5.0,12.0,0.001,0.9661
4,4.0,4.0,12.0,0.001,0.9611


In [ ]:
Tr_result_temp_pd.to_csv('Tr_result.csv')

In [32]:
a = pd.DataFrame(model.predict(TestData))

In [33]:
np.where(a != 1.0)

(array([   0,    0,    0, ..., 9999, 9999, 9999]),
 array([0, 1, 2, ..., 7, 8, 9]))